In [59]:
from tkinter import *
from functools import partial
from Census21_CACD_Wrapper import APIWrapper
FONT = ("arial",10)

In [60]:
class Interface():
    def __init__(self):
        self.configure_api()
        self.configure_window()
        self.start_selections()
        
        # Define variables
        self.dimension_drop_1 = None
        self.dimension_drop_2 = None
        self.regional_drop = None
        
    
    def configure_api(self):
        '''
        Configures API for use
        '''
        self.api = APIWrapper()
        return self.api
    
    
    def configure_window(self):
        '''
        Configure window for interface usage
        '''
        self.window = Tk()
        self.window.title('Census API')
        self.window.configure(padx=50, pady=50)


    def drop_config_button(self, text, selection, row, column, command):
        '''
        Create a dropdown with a button

        Args:
            text: str = The label for dropdown
            selection: iterable object =  The options for the dropdown
            row: int = The row to place the dropdown
            colum: int = The column to place the dropdown
            command: function = The command to execute when the button is clicked

        Returns:
            dropdown list as object
        '''
        label = Label(text=text, font=FONT)
        label.grid(row=row, column=column)

        variable = StringVar(self.window)
        drop_list = OptionMenu(self.window, variable, *selection)
        variable.set(text)
        drop_list.grid(row=row+1, column=column)

        residence_select = Button(text="Select", command=command)
        residence_select.grid(row=row+2, column=column)

        return drop_list


    def drop_config(self, text, selection, row, column):
        '''
         Create a dropdown without a button

        Args:
            text: str = The label for dropdown
            selection: iterable object =  The options for the dropdown
            row: int = The row to place the dropdown
            colum: int = The column to place the dropdown

        Returns:
             dropdown list as object
        '''
        label = Label(text=text, font=FONT)
        label.grid(row=row, column=column)

        variable = StringVar(self.window)
        drop_list = OptionMenu(self.window, variable, *selection)
        variable.set(text)
        drop_list.grid(row=row+1, column=column)

        return drop_list


    def retrieve_res_input(self):
        '''
        Retrieve the selection of 'residence dropdown list' and returns abbreviation (abbreviation used in API call)
        '''
        output = self.residence_drop.cget('text')
        if output == 'Usual Residence':
            self.residence_code = 'UR'
        elif output == 'Household':
            self.residence_code = 'HH'
        else:
            self.residence_code = 'HRP'

        dimension_list = self.list_dimensions()
        self.create_dimension_selection(dimension_list)


    def execute_api_call(self):
        '''
        Query API and save data to csv file in '/data/output/'

        Args:
            residence_code: tkinter dropdown object 
            dimension_drop_1: tkinter dropdown object 
            Dimension_drop_2: tkinter dropdown object 
            regional drop: tkinter dropdown object 

        Returns:
             Saves Data to .csv
        '''
        dim_1 = self.dimension_drop_1.cget('text')
        dim_2 = self.dimension_drop_2.cget('text')
        region_dict = self.api.get_areas_by_pop_type(self.residence_code)
        region_value = self.regional_drop.cget('text')
        region = region_dict[region_value]
        data = self.api.query_api(self.residence_code, f'{dim_1},{dim_2}', region)
        data.to_csv(f'data/output/{self.residence_code}_{dim_1}_{dim_2}_{region}.csv')
        print(data)


    def create_dimension_selection(self, dimension_list):
        '''
        Creates the remaining three selection dropdown lists & 'Save Data' button after residence selected
        '''
        # Retrieve list of dimensions 
        self.dimension_drop_1 = self.drop_config('Variable 1', dimension_list, 
                                 row=2, column=2) 
        self.dimension_drop_2 = self.drop_config('Variable 2', dimension_list, 
                                 row=2, column=3)
        area_codes = self.list_areas()
        self.regional_drop = self.drop_config('Regional Divisions', 
                                                area_codes, row=2, column=4)
        but = Button( text = 'Save Data', command=partial(self.execute_api_call))
        but.grid(column=4, row=4)


    def list_dimensions(self):
        '''
        Returns the list of demensions relating to that residence code (HH, HRP, UR) & add option for None
        '''
        dimension_list = list(self.api.get_dims_by_pop_type(self.residence_code).values())
        dimension_list.append('None')
        return dimension_list


    def list_areas(self):
        '''
        returns a list of areas relating to the selected residence code
        '''
        area_list = list(self.api.get_areas_by_pop_type(self.residence_code).keys())
        return area_list


    def start_selections(self):
        '''
        Creates Selection drop down
        '''
        self.residence_drop = self.drop_config_button('Residence Selection', ('Usual Residence', 'Household', 'Household Reference Person'), 
                                 row=2, column=1, command=self.retrieve_res_input)
        return self.residence_drop

    
    
interface = Interface()



# Keep window running
window.mainloop()

       ctry health_in_general  \
0   England    Does not apply   
1   England    Does not apply   
2   England    Does not apply   
3   England    Does not apply   
4   England    Does not apply   
..      ...               ...   
91    Wales   Very bad health   
92    Wales   Very bad health   
93    Wales   Very bad health   
94    Wales   Very bad health   
95    Wales   Very bad health   

                                highest_qualification  count  
0                                      Does not apply      0  
1                                   No qualifications      0  
2   Level 1 and entry level qualifications: 1 to 4...      0  
3   Level 2 qualifications: 5 or more GCSEs (A* to...      0  
4                                      Apprenticeship      0  
..                                                ...    ...  
91  Level 2 qualifications: 5 or more GCSEs (A* to...   4844  
92                                     Apprenticeship   3597  
93  Level 3 qualifications: 2 or mor